In [1]:
'''
參考https://github.com/Asoul/tsec
'''
## -*- coding: utf-8 -*-
#-*- coding: cp950 -*-
import os
import re
import csv
import logging
import requests
import time
from datetime import datetime, timedelta
'''
定義Fuction
'''

#清除抓到的股票每個欄位中多餘的逗號還有空白
def clean_row(row):
    ''' Clean comma and spaces '''
    for index, content in enumerate(row):
        row[index] = re.sub(",", "", content.strip())
    return row

#紀錄stock_id這隻股票的資料以append的方式增加在stock_id.csv後面
def record(stock_id, row):
    ''' Save row to csv file '''
    prefix="data"
    f = open('../{}/{}.csv'.format(prefix, stock_id.strip()), 'a')
    cw = csv.writer(f, lineterminator='\n')
    cw.writerow(row)
    f.close()
#初始化csv檔的column
def Initialize(Stock_ID):
    #初始化Stock_ID.csv
    prefix = "../data"
    if not os.path.isfile('{}/{}.csv'.format(prefix, Stock_ID.strip())):
        ''' Save row to csv file '''
        f = open('{}/{}.csv'.format(prefix, Stock_ID.strip()), 'a', encoding='utf-8-sig')
        cw = csv.writer(f, lineterminator='\n')
        csv_Columns = ['日期','成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數','最後揭示買價','最後揭示賣價',]
        cw.writerow(csv_Columns)
        f.close()

In [2]:
#爬Stock_ID這支股票指定日期Day這天的資料
def Get_TSEdata(Day, Stock_ID):
    #設定要開始爬的日期轉成民國，再設定網頁中要輸入的選項
    Date_str = '{0}{1:02d}{2:02d}'.format(Day.year, Day.month, Day.day)
    Store_time = '{0}-{1:02d}-{2:02d}'.format(Day.year, Day.month, Day.day)
    #print(Date_str)
    url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX'
    query_params = {
        'date': Date_str,
        'response': 'json',
        'type': 'ALL',
        '_': str(round(time.time() * 1000) - 500)
    }
    # Get json data
    page = requests.get(url, params=query_params)
    for ID in Stock_ID:
        Initialize(ID)
    if not page.ok:
        logging.error("Can not get TSE data at {}".format(Date_str))
    else:
        # Parse page
        content = page.json()
        #print(content)
        for attri in content:
            #print(attri)
            if attri == 'data5' or attri == 'data4' or attri == 'data3' or  attri == 'data2' or attri == 'data1':
                for data in content[attri]:
                    for ID in Stock_ID:
                        if str(data[0]).strip() == ID:
                            sign = '-' if data[9].find('green') > 0 else ''
                            #print(data)
                            row = [
                                Store_time, # 日期
                                data[2], # 成交股數
                                data[4], # 成交金額
                                data[5], # 開盤價
                                data[6], # 最高價
                                data[7], # 最低價
                                data[8], # 收盤價
                                sign + data[10], # 漲跌價差
                                data[3], # 成交筆數
                                data[11],#最後揭示買價
                                data[13],#最後揭示賣價
                            ]
                            #print(row)
                            clean_row(row)
                            print(row)
                            record(data[0], row)

'''
抓取Stock_ID在日期範圍(First_Day,Last_Day)內的所有資料
['日期','成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數']
'''
#預設抓台積電(2330)從今天到2004,2,11的資料
def Get_Stock_DATA(Stock_ID = ["2330"], First_Day = datetime.today(), Last_Day = datetime(2004,2,11)):
    #Set Stock_ID that need to crawl
    print("Crawl " + str(Stock_ID) + "Stock Data")
    #Set logging
    if not os.path.isdir('../log'):
        os.makedirs('../log')
    logging.basicConfig(filename='../log/crawl-error.log',
                        level=logging.ERROR,
                        format='%(asctime)s\t[%(levelname)s]\t%(message)s',
                        datefmt='%Y/%m/%d %H:%M:%S')
    #Make directory if not exist when initialize
    prefix='../data'
    if not os.path.isdir(prefix):
        os.mkdir(prefix)
        
    #The First_Day and Last_Day
    Date_str = '{0}/{1:02d}/{2:02d}'.format(First_Day.year - 1911, First_Day.month, First_Day.day)
    CE_Date_str = '{0}/{1:02d}/{2:02d}'.format(First_Day.year, First_Day.month, First_Day.day)
    print("Start on " + Date_str)
    print("Start on " + CE_Date_str)
    Date_str = '{0}/{1:02d}/{2:02d}'.format(Last_Day.year - 1911, Last_Day.month, Last_Day.day)
    CE_Date_str = '{0}/{1:02d}/{2:02d}'.format(Last_Day.year, Last_Day.month, Last_Day.day)
    print("End of " + Date_str)
    print("End of " + CE_Date_str)
    
    #Set Max_Error to 5
    Max_Error = 5
    Error_Times = 0
    #Crawl stock until Last_Day
    while Error_Times < Max_Error and First_Day >= Last_Day:
        try:
            Get_TSEdata(Last_Day, Stock_ID)
            Error_Times = 0
        except:
            '''When crawl data occuring problem add one to Error_Times'''
            date_str = Last_Day.strftime('%Y/%m/%d')
            logging.error('Crawl raise error {}'.format(date_str))
            Error_Times += 1
            continue
        finally:
            Last_Day += timedelta(1)
            
    print("Finish")

In [3]:
Stock_ID = ["2330","2002","3008",'2332']
#print(len(Stock_ID))
#Stock_ID = ["2332"]
#datetime(2004,2,11) data2
#datetime(2009,1,5) data4
#datetime(2011,8,1) data5
#Get_Stock_DATA(Stock_ID, Last_Day = datetime.today() - timedelta(10))
Get_Stock_DATA(Stock_ID=Stock_ID)
#Get_TSEdata(datetime(2009,1,5), ['3008'])

Crawl ['2330', '2002', '3008', '2332']Stock Data
Start on 106/06/07
Start on 2017/06/07
End of 93/02/11
End of 2004/02/11
['2004-02-11', '52895919', '1600114397', '30.60', '30.60', '30.00', '30.10', '-0.30', '7439', '30.10', '30.20']
['2004-02-11', '18589704', '1188570617', '64.50', '64.50', '63.50', '64.00', '0.00', '4055', '63.50', '64.00']
['2004-02-11', '6339088', '290089861', '45.80', '46.30', '45.40', '45.50', '-0.40', '1322', '45.50', '45.60']
['2004-02-11', '554715', '200108733', '361.00', '363.00', '357.00', '357.00', '-1.00', '365', '357.00', '358.00']
['2004-02-12', '77885197', '2362843009', '30.20', '30.60', '30.10', '30.30', '0.20', '8399', '30.30', '30.40']
['2004-02-12', '48648900', '3136959844', '64.00', '65.00', '64.00', '64.50', '0.50', '6758', '64.00', '64.50']
['2004-02-12', '15800364', '702475068', '45.50', '45.50', '44.00', '44.00', '-1.50', '3871', '44.00', '44.20']
['2004-02-12', '758347', '271993464', '359.00', '362.00', '356.00', '356.00', '-1.00', '477', '356

['2004-03-09', '45031607', '1542670507', '34.00', '34.50', '34.00', '34.50', '0.30', '6860', '34.40', '34.50']
['2004-03-09', '36685082', '2393354776', '65.00', '66.00', '64.50', '65.50', '-0.50', '6553', '65.50', '66.00']
['2004-03-09', '3174434', '131795959', '41.30', '41.90', '41.30', '41.40', '0.10', '970', '41.40', '41.50']
['2004-03-09', '684000', '269302000', '392.00', '398.00', '389.00', '395.00', '5.00', '411', '393.00', '395.00']
['2004-03-10', '52276822', '1796439817', '34.50', '34.80', '34.10', '34.20', '-0.30', '6609', '34.20', '34.30']
['2004-03-10', '87607498', '5591609958', '65.00', '65.00', '63.00', '63.00', '-2.50', '18128', '63.00', '63.50']
['2004-03-10', '4407620', '182755601', '41.60', '41.90', '41.20', '41.30', '-0.10', '1166', '41.30', '41.40']
['2004-03-10', '952230', '369264394', '395.00', '395.00', '383.00', '383.00', '-12.00', '645', '383.00', '384.00']
['2004-03-11', '73415753', '2496953407', '34.00', '34.30', '33.80', '34.10', '-0.10', '8810', '34.10', '34

['2004-04-05', '53557988', '1828455521', '34.10', '34.40', '33.90', '34.30', '0.50', '7534', '34.20', '34.30']
['2004-04-05', '59968347', '3662412969', '61.00', '62.00', '60.00', '62.00', '2.50', '13678', '61.50', '62.00']
['2004-04-05', '11069872', '465044465', '41.50', '42.60', '41.20', '42.20', '1.20', '2376', '42.20', '42.30']
['2004-04-05', '557000', '220009000', '396.00', '398.00', '391.00', '397.00', '4.00', '363', '396.00', '397.00']
['2004-04-06', '75457768', '2591743366', '34.40', '34.70', '34.10', '34.10', '-0.20', '8083', '34.00', '34.10']
['2004-04-06', '47067612', '2901941398', '62.50', '62.50', '61.00', '62.00', '0.00', '8605', '61.50', '62.00']
['2004-04-06', '4478655', '187893451', '42.20', '42.20', '41.60', '42.00', '-0.20', '1194', '41.90', '42.00']
['2004-04-06', '392550', '155556253', '398.00', '398.00', '392.00', '395.00', '-2.00', '309', '395.00', '397.00']
['2004-04-07', '44361623', '1515842716', '34.20', '34.40', '34.00', '34.40', '0.30', '4700', '34.30', '34.4

['2004-04-30', '173205116', '5108272172', '30.50', '30.50', '29.00', '29.60', '-1.30', '19031', '29.60', '29.70']
['2004-04-30', '86620034', '5021998441', '60.00', '60.00', '56.50', '57.50', '-3.00', '18937', '57.50', '58.00']
['2004-04-30', '31781220', '1406785261', '45.00', '45.20', '42.40', '44.70', '1.20', '8674', '44.70', '44.80']
['2004-04-30', '747360', '281356910', '382.00', '386.00', '370.00', '372.00', '-10.00', '580', '372.00', '375.00']
['2004-05-03', '112736183', '3236976983', '29.20', '29.40', '28.10', '28.50', '-1.10', '16215', '28.50', '28.60']
['2004-05-03', '33093691', '1909918904', '57.50', '58.50', '57.00', '58.00', '0.50', '8387', '57.50', '58.00']
['2004-05-03', '8624145', '379500383', '45.00', '45.00', '43.00', '43.50', '-1.20', '2350', '43.50', '43.60']
['2004-05-03', '572200', '212837027', '372.00', '378.00', '368.00', '372.00', '0.00', '419', '370.00', '372.00']
['2004-05-04', '85606525', '2525324627', '28.80', '30.40', '28.80', '30.10', '1.60', '12554', '30.0

['2004-05-27', '28163511', '845366740', '30.10', '30.20', '29.80', '30.10', '0.30', '4326', '30.00', '30.10']
['2004-05-27', '35343089', '2006143156', '57.00', '57.00', '56.00', '57.00', '1.00', '7266', '56.50', '57.00']
['2004-05-27', '2932799', '124236421', '42.40', '42.70', '41.90', '42.70', '0.70', '1126', '42.60', '42.70']
['2004-05-27', '295131', '114452310', '392.00', '392.00', '385.00', '390.00', '4.00', '255', '389.00', '390.00']
['2004-05-28', '44295138', '1331631272', '30.30', '30.30', '29.80', '30.30', '0.20', '5358', '30.20', '30.30']
['2004-05-28', '51381517', '2959538145', '57.50', '58.00', '57.00', '58.00', '1.00', '8826', '57.50', '58.00']
['2004-05-28', '3368413', '144737179', '42.80', '43.20', '42.70', '43.10', '0.40', '1394', '43.00', '43.10']
['2004-05-28', '286078', '111511316', '391.00', '392.00', '387.00', '390.00', '0.00', '225', '388.00', '390.00']
['2004-05-31', '24169866', '721597200', '30.10', '30.10', '29.60', '29.60', '-0.70', '4190', '29.60', '29.70']
['

...

['2017-01-19', '16550442', '414760900', '25.05', '25.20', '24.80', '25.10', '0.00', '4856', '25.10', '25.15']
['2017-01-19', '25935613', '4673783621', '179.50', '181.00', '179.50', '180.50', '-0.50', '8400', '180.00', '180.50']
['2017-01-19', '1070211', '11302458', '10.50', '10.60', '10.45', '10.55', '0.10', '419', '10.50', '10.55']
['2017-01-19', '772638', '3346704260', '4410.00', '4420.00', '4275.00', '4295.00', '-150.00', '877', '4295.00', '4300.00']
['2017-01-20', '11339284', '284162400', '25.00', '25.10', '24.95', '25.10', '0.00', '4008', '25.05', '25.10']
['2017-01-20', '42317086', '7658563990', '181.00', '181.50', '180.50', '181.00', '0.50', '7803', '181.00', '181.50']
['2017-01-20', '309667', '3259299', '10.50', '10.60', '10.50', '10.50', '-0.05', '171', '10.50', '10.55']
['2017-01-20', '818892', '3539696650', '4280.00', '4390.00', '4220.00', '4390.00', '95.00', '769', '4380.00', '4390.00']
['2017-01-23', '17919982', '452097021', '25.20', '25.30', '25.15', '25.25', '0.15', '499

['2017-02-21', '17762283', '463369808', '26.10', '26.20', '26.00', '26.10', '0.15', '7658', '26.05', '26.10']
['2017-02-21', '31987532', '6058373156', '190.00', '190.00', '188.50', '190.00', '0.00', '8646', '189.50', '190.00']
['2017-02-21', '4715268', '53595409', '11.25', '11.60', '11.15', '11.15', '-0.05', '1824', '11.15', '11.20']
['2017-02-21', '235602', '1068287160', '4515.00', '4560.00', '4495.00', '4560.00', '65.00', '299', '4550.00', '4565.00']
['2017-02-22', '16305736', '425343886', '26.15', '26.20', '26.05', '26.10', '0.00', '5893', '26.10', '26.15']
['2017-02-22', '26144905', '4946742296', '190.50', '191.00', '188.50', '188.50', '-1.50', '6199', '188.50', '189.00']
['2017-02-22', '5040211', '59188022', '11.25', '11.25', '11.10', '11.20', '0.05', '869', '11.15', '11.20']
['2017-02-22', '605530', '2832424500', '4640.00', '4730.00', '4635.00', '4680.00', '120.00', '688', '4680.00', '4690.00']
['2017-02-23', '13161341', '342757966', '26.15', '26.15', '25.95', '26.10', '0.00', '4

['2017-03-21', '14269104', '368372527', '25.80', '25.90', '25.75', '25.90', '0.05', '5962', '25.85', '25.90']
['2017-03-21', '33662234', '6523324906', '192.00', '195.00', '191.50', '195.00', '3.50', '13192', '194.50', '195.00']
['2017-03-21', '29743426', '417603654', '13.90', '14.40', '13.65', '14.20', '0.55', '8373', '14.15', '14.20']
['2017-03-21', '413832', '1990059262', '4750.00', '4850.00', '4750.00', '4810.00', '90.00', '490', '4805.00', '4810.00']
['2017-03-22', '11475745', '295485233', '25.80', '25.85', '25.65', '25.75', '-0.15', '4525', '25.75', '25.80']
['2017-03-22', '27766559', '5349879407', '192.50', '193.50', '191.50', '193.50', '-1.50', '9435', '193.00', '193.50']
['2017-03-22', '25021261', '364057725', '13.95', '15.40', '13.80', '15.20', '1.00', '7610', '15.20', '15.25']
['2017-03-22', '411704', '1946741960', '4725.00', '4765.00', '4685.00', '4740.00', '-70.00', '498', '4735.00', '4750.00']
['2017-03-23', '9221752', '236801987', '25.80', '25.85', '25.60', '25.70', '-0.0

['2017-04-18', '10290034', '257588413', '25.00', '25.10', '25.00', '25.00', '0.00', '5945', '25.00', '25.05']
['2017-04-18', '28732770', '5411365190', '188.00', '189.50', '187.50', '188.00', '0.50', '6174', '187.50', '188.00']
['2017-04-18', '5665003', '70901469', '12.25', '12.80', '12.15', '12.50', '0.45', '1845', '12.50', '12.55']
['2017-04-18', '443633', '2122257575', '4810.00', '4810.00', '4765.00', '4780.00', '0.00', '315', '4780.00', '4795.00']
['2017-04-19', '42473827', '1037596125', '25.00', '25.00', '24.20', '24.20', '-0.80', '24052', '24.20', '24.25']
['2017-04-19', '31171199', '5828201425', '187.50', '187.50', '186.50', '186.50', '-1.50', '9427', '186.50', '187.00']
['2017-04-19', '3668411', '44677711', '12.40', '12.40', '12.00', '12.05', '-0.45', '1500', '12.05', '12.10']
['2017-04-19', '309115', '1465729350', '4745.00', '4760.00', '4720.00', '4720.00', '-60.00', '437', '4715.00', '4720.00']
['2017-04-20', '30470928', '730310972', '24.20', '24.20', '23.80', '23.85', '-0.35'

['2017-05-15', '8817838', '212264031', '23.95', '24.15', '23.95', '24.15', '0.25', '3627', '24.10', '24.15']
['2017-05-15', '25922334', '5324070590', '204.00', '206.00', '204.00', '206.00', '0.00', '8141', '205.50', '206.00']
['2017-05-15', '3876064', '45576233', '12.10', '12.10', '11.65', '11.65', '-0.35', '1209', '11.65', '11.70']
['2017-05-15', '227020', '1110453000', '4900.00', '4915.00', '4855.00', '4900.00', '15.00', '330', '4895.00', '4910.00']
['2017-05-16', '32933836', '807891835', '24.60', '24.65', '24.40', '24.65', '0.50', '11917', '24.60', '24.65']
['2017-05-16', '36604672', '7468111624', '205.00', '205.00', '203.50', '204.50', '-1.50', '11066', '204.00', '204.50']
['2017-05-16', '3540932', '40659063', '11.75', '11.75', '11.35', '11.40', '-0.25', '1205', '11.35', '11.40']
['2017-05-16', '255839', '1251588515', '4920.00', '4925.00', '4870.00', '4870.00', '-30.00', '339', '4870.00', '4895.00']
['2017-05-17', '21660009', '532274243', '24.65', '24.75', '24.40', '24.75', '0.10',